In [10]:
import paramiko
import threading

class ServerThread(threading.Thread):
    def __init__(self, hostname, username, key_filename, command):
        threading.Thread.__init__(self)
        self.hostname = hostname
        self.username = username
        self.key_filename = key_filename
        self.command = command

    def run(self):
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        try:
            client.connect(hostname=self.hostname, username=self.username, key_filename=self.key_filename)
            stdin, stdout, stderr = client.exec_command(self.command)
            stdout.channel.set_combine_stderr(True)
            output = stdout.readlines()
            print(f"Output from {self.hostname}:\n{''.join(output)}")
        except Exception as e:
            print(f"Connection Failed for {self.hostname}: {e}")
        finally:
            client.close()

In [13]:
# List of servers
servers = [
    {"hostname": "192.168.10.91", "username": "root", "key_filename": "/home/jsh/.ssh/ten-jsh.pem"},
    {"hostname": "192.168.10.101", "username": "root", "key_filename": "/home/jsh/.ssh/ten-jsh.pem"},
    {"hostname": "192.168.10.111", "username": "root", "key_filename": "/home/jsh/.ssh/ten-jsh.pem"},
]

# Command to run
command = "ctr -n k8s.io image rm docker.io/asdfry/dali-example:imagenet"  # Example command

threads = []

# Create and start threads for each server
for server in servers:
    thread = ServerThread(server["hostname"], server["username"], server["key_filename"], command)
    thread.start()
    threads.append(thread)

# Wait for all threads to complete
for t in threads:
    t.join()

Output from 192.168.10.91:
docker.io/asdfry/dali-example@sha256:849046a4158e06d13a17b940c53b73574f3db7787a2397774b2d2beb4860aa02

Output from 192.168.10.101:
docker.io/asdfry/dali-example@sha256:849046a4158e06d13a17b940c53b73574f3db7787a2397774b2d2beb4860aa02

Output from 192.168.10.111:
docker.io/asdfry/dali-example@sha256:849046a4158e06d13a17b940c53b73574f3db7787a2397774b2d2beb4860aa02

